# Step1. 관련 패키지 및 모듈 import 하기

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from torch.nn import init

# Step 2. 와인 데이터셋 구성하기

In [ ]:
train_data = pd.read_csv('drive/My Drive/Colab Notebooks/deepLearningCodes/data/winequality_train_all_shuffle.csv', sep=',')
test_data = pd.read_csv('drive/My Drive/Colab Notebooks/deepLearningCodes/data/winequality_test_all.csv', sep=',')

# Generate Test Label
test_label = pd.DataFrame([0. if item == '0' else 1. for item in test_data['class']])
test_label = [0. if item == 0 else 1. for item in test_data['class']]

# drop the class infromation
test_data = test_data.drop(['class'], axis=1)

# Step 3. 다층퍼셉트론 모델 설계하기

In [ ]:
class mymodel(nn.Module):
    def __init__(self, is_train=True):        
        super(mymodel, self).__init__()
        ################################
        ## Problem #1        
        ################################
                        
    def forward(self, x):        
        ################################
        ## Problem #1        
        ################################        
        return x

# 모델 구조 확인하기

In [ ]:
def print_network(model):    
    def _get_network_description(network):
        '''Get the string and total parameters of the network'''
        if isinstance(network, nn.DataParallel):
            network = network.module
        s = str(network)
        n = sum(map(lambda x: x.numel(), network.parameters()))
        return s, n
    s, n = _get_network_description(model)
    if isinstance(model, nn.DataParallel):
        net_struc_str = '{} - {}'.format(model.__class__.__name__, model.module.__class__.__name__)
    else: net_struc_str = '{}'.format(model.__class__.__name__)
    log = 'Network structure: {}, with parameters: {:,d}'.format(net_struc_str, n)
    return log, s

model = mymodel()
log, architecture = print_network(model)
print(log)
print(architecture)

Network structure: mymodel, with parameters: 0
mymodel()


# Step 4. Training 함수 구현하기

In [ ]:
def train(train_data, test_data, test_label, batch_size=1000, epochs=1000, learning_rate = 5e-4):
    # training on cpu
    device = torch.device('cpu')
    # random seed
    torch.manual_seed(1)
    # define model
    model = mymodel().to(device)
    # optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0, betas=(0.9, 0.999))        
    # loss fucntion
    criterion = nn.CrossEntropyLoss().to(device)
    
    # now start Training!!!
    running_loss = 0.0
    for epoch in range(epochs):
        model.train()

        # shuffling every epoch        
        train_data_shuffle = train_data
        train_label_shuffle = pd.DataFrame([0. if item == '0' else 1. for item in train_data_shuffle['class']])
        train_label_shuffle = [0. if item == 0 else 1. for item in train_data_shuffle['class']]

        train_data_shuffle = train_data_shuffle.drop(['class'], axis=1)


        for start, end in zip(range(0, len(train_data_shuffle), batch_size), range(batch_size, len(train_data_shuffle), batch_size)):       

            inputs = torch.from_numpy(train_data_shuffle[start:end].as_matrix()).to(device)
            inputs = inputs.float()
            labels = torch.Tensor(train_label_shuffle[start:end]).to(device)
            labels = labels.long()
                 
            outputs = model(inputs)            
            loss = criterion(outputs, labels)
            
            ######################################################
            ### Problem 2: Backpropagation, Updating Model Weight
            ######################################################
            
            # print statistics
            running_loss += loss.item()
        
              
        total = 0
        correct = 0
        if epoch % 50 == 0 or epoch +1 == epochs:
            model.eval()
            with torch.no_grad(): 
                total = len(test_label)
                test_outputs = model(Variable(torch.from_numpy(test_data.as_matrix()).float()).to(device))               
                
                ###################################################
                #### Problem 3: Test model every 50 epochs
                ###################################################                
                                
                print('[Epoch: {}] [Training Loss: {:.6f}] [Accuracy: {:.6f}]'.format(epoch, running_loss / batch_size, accuracy))
                running_loss = 0.0
        model.train()
    print("End Training!")

In [ ]:
train(train_data, test_data, test_label)

ValueError: optimizer got an empty parameter list